In [1]:
import pandas as pd
import numpy as np

path = "C:\\Users\\shash\\Desktop\\AB testing\\retail_paid_vs_organic_ugc_experiment.csv"

df = pd.read_csv(path, parse_dates=[
    "first_purchase_date", "delivery_date", "ugc_email_sent_date"
])

print("Shape:", df.shape)
print("Columns:", df.columns.tolist())

print("\nDuplicate user_id count:", df["user_id"].duplicated().sum())
print("\nAcquisition channel counts:\n", df["acquisition_channel"].value_counts())
print("\nVariant counts:\n", df["variant"].value_counts())

cols_to_check = [
    "ugc_email_sent_date", "ugc_email_opened", "ugc_email_clicked",
    "repeat_purchase_within_30d", "time_to_next_purchase_days",
    "repeat_order_value"
]
print("\nMissingness:\n", df[cols_to_check].isna().mean().sort_values(ascending=False))

paid = df[df["acquisition_channel"] == "paid"].copy()
organic = df[df["acquisition_channel"] == "organic"].copy()

paid_ctrl = paid[paid["variant"] == "control"].copy()
paid_trt  = paid[paid["variant"] == "treatment"].copy()

print("\nPaid cohort size:", paid.shape[0])
print("  Paid control:", paid_ctrl.shape[0])
print("  Paid treatment:", paid_trt.shape[0])
print("Organic benchmark size:", organic.shape[0])

def summarize_balance(paid_ctrl, paid_trt):
    out = {}
    # numeric
    for col in ["first_order_value", "returned_within_14d"]:
        out[col] = {
            "control_mean": paid_ctrl[col].mean(),
            "treat_mean": paid_trt[col].mean(),
            "diff": paid_trt[col].mean() - paid_ctrl[col].mean()
        }
    return pd.DataFrame(out).T

balance_num = summarize_balance(paid_ctrl, paid_trt)
print("\nNumeric balance (Paid control vs treatment):\n", balance_num)


cat_cols = ["category", "region"]
for c in cat_cols:
    tab = pd.crosstab(paid[c], paid["variant"], normalize="columns")
    print(f"\nDistribution by {c} (normalized within variant):\n", tab)

# a) Control should have no email sent (NaT)
ctrl_sent_rate = paid_ctrl["ugc_email_sent_date"].notna().mean()
trt_sent_rate  = paid_trt["ugc_email_sent_date"].notna().mean()
print("\nUGC email sent rate:")
print("  Control sent rate:", ctrl_sent_rate)
print("  Treatment sent rate:", trt_sent_rate)

# b) Treatment email should be sent AFTER delivery and within 5-7 days
paid_trt["days_after_delivery"] = (paid_trt["ugc_email_sent_date"] - paid_trt["delivery_date"]).dt.days

print("\nTreatment days_after_delivery summary:")
print(paid_trt["days_after_delivery"].describe())

# check invalids
invalid_before_delivery = (paid_trt["days_after_delivery"] < 0).sum()
invalid_outside_window = (~paid_trt["days_after_delivery"].between(5, 7)).sum()

print("\nInvalid timing counts:")
print("  Emails sent before delivery:", invalid_before_delivery)
print("  Emails outside 5-7 day window:", invalid_outside_window)

C:\Users\shash\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Shape: (20000, 16)
Columns: ['user_id', 'acquisition_channel', 'variant', 'first_purchase_date', 'delivery_date', 'ugc_email_sent_date', 'ugc_email_opened', 'ugc_email_clicked', 'category', 'region', 'first_order_value', 'returned_within_14d', 'repeat_purchase_within_30d', 'time_to_next_purchase_days', 'repeat_order_value', 'discount_used_on_repeat']

Duplicate user_id count: 0

Acquisition channel counts:
 acquisition_channel
paid       12000
organic     8000
Name: count, dtype: int64

Variant counts:
 variant
benchmark    8000
control      6033
treatment    5967
Name: count, dtype: int64

Missingness:
 time_to_next_purchase_days    0.84065
repeat_order_value            0.84065
ugc_email_sent_date           0.70165
ugc_email_opened              0.00000
ugc_email_clicked             0.00000
repeat_purchase_within_30d    0.00000
dtype: float64

Paid cohort size: 12000
  Paid control: 6033
  Paid treatment: 5967
Organic benchmark size: 8000

Numeric balance (Paid control vs treatment):
 

In [2]:
from statsmodels.stats.proportion import proportions_ztest
import numpy as np

# Paid cohorts
paid_ctrl = df[(df.acquisition_channel == "paid") & (df.variant == "control")]
paid_trt  = df[(df.acquisition_channel == "paid") & (df.variant == "treatment")]


ctrl_n = paid_ctrl.shape[0]
trt_n  = paid_trt.shape[0]

ctrl_success = paid_ctrl["repeat_purchase_within_30d"].sum()
trt_success  = paid_trt["repeat_purchase_within_30d"].sum()


ctrl_rate = ctrl_success / ctrl_n
trt_rate  = trt_success / trt_n


abs_lift_pp = (trt_rate - ctrl_rate) * 100
rel_lift_pct = (trt_rate - ctrl_rate) / ctrl_rate * 100

# Z-test
count = np.array([trt_success, ctrl_success])
nobs  = np.array([trt_n, ctrl_n])

z_stat, p_value = proportions_ztest(count, nobs)

# 95% CI for difference in proportions
from statsmodels.stats.proportion import confint_proportions_2indep

ci_low, ci_high = confint_proportions_2indep(
    trt_success, trt_n,
    ctrl_success, ctrl_n,
    method="wald"
)

print("Paid Control Repeat Rate:", round(ctrl_rate, 4))
print("Paid Treatment Repeat Rate:", round(trt_rate, 4))
print("Absolute Lift (pp):", round(abs_lift_pp, 2))
print("Relative Lift (%):", round(rel_lift_pct, 2))
print("\nZ-stat:", round(z_stat, 3))
print("P-value:", round(p_value, 6))
print("95% CI for lift (pp):", (round(ci_low*100, 2), round(ci_high*100, 2)))


Paid Control Repeat Rate: 0.0892
Paid Treatment Repeat Rate: 0.1483
Absolute Lift (pp): 5.91
Relative Lift (%): 66.32

Z-stat: 10.019
P-value: 0.0
95% CI for lift (pp): (4.76, 7.07)


In [3]:
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.proportion import confint_proportions_2indep
from scipy import stats

paid_ctrl = df[(df.acquisition_channel == "paid") & (df.variant == "control")].copy()
paid_trt  = df[(df.acquisition_channel == "paid") & (df.variant == "treatment")].copy()

def prop_test(success_a, n_a, success_b, n_b, label=""):
    """Compare proportion A vs B using z-test and CI for diff (A-B)."""
    count = np.array([success_a, success_b])
    nobs  = np.array([n_a, n_b])
    z, p = proportions_ztest(count, nobs)
    ci_low, ci_high = confint_proportions_2indep(success_a, n_a, success_b, n_b, method="wald")
    diff_pp = (success_a/n_a - success_b/n_b) * 100
    print(f"\n{label}")
    print(f"A rate: {success_a/n_a:.4f}  | B rate: {success_b/n_b:.4f}")
    print(f"Diff (A-B) pp: {diff_pp:.2f}")
    print(f"95% CI (pp): ({ci_low*100:.2f}, {ci_high*100:.2f})")
    print(f"z={z:.3f}, p={p:.6f}")

# Guardrail: Return rate
# -------------------------
ctrl_ret = paid_ctrl["returned_within_14d"].sum()
trt_ret  = paid_trt["returned_within_14d"].sum()

prop_test(
    success_a=trt_ret, n_a=paid_trt.shape[0],
    success_b=ctrl_ret, n_b=paid_ctrl.shape[0],
    label="4.1 Guardrail — Return within 14d (Treatment vs Control)"
)

# Time to next purchase (repeaters only)
# -------------------------
ctrl_rep = paid_ctrl[paid_ctrl["repeat_purchase_within_30d"] == 1].copy()
trt_rep  = paid_trt[paid_trt["repeat_purchase_within_30d"] == 1].copy()

print("\n4.2 Time to next purchase (days) — repeaters only")
print("Control repeaters n:", ctrl_rep.shape[0], "Treatment repeaters n:", trt_rep.shape[0])

# Summary stats
def summarize_num(a, b, col):
    out = pd.DataFrame({
        "group": ["control", "treatment"],
        "n": [a[col].notna().sum(), b[col].notna().sum()],
        "mean": [a[col].mean(), b[col].mean()],
        "median": [a[col].median(), b[col].median()],
        "p25": [a[col].quantile(0.25), b[col].quantile(0.25)],
        "p75": [a[col].quantile(0.75), b[col].quantile(0.75)],
    })
    return out

time_summary = summarize_num(ctrl_rep, trt_rep, "time_to_next_purchase_days")
print(time_summary)

# Non-parametric test (robust)
mw = stats.mannwhitneyu(trt_rep["time_to_next_purchase_days"].dropna(),
                        ctrl_rep["time_to_next_purchase_days"].dropna(),
                        alternative="two-sided")
print("Mann–Whitney U p-value:", mw.pvalue)

# -------------------------
# 4.3 Secondary: Repeat order value (repeaters only)
# -------------------------
print("\n4.3 Repeat order value — repeaters only")
value_summary = summarize_num(ctrl_rep, trt_rep, "repeat_order_value")
print(value_summary)

mw_val = stats.mannwhitneyu(trt_rep["repeat_order_value"].dropna(),
                            ctrl_rep["repeat_order_value"].dropna(),
                            alternative="two-sided")
print("Mann–Whitney U p-value:", mw_val.pvalue)

# -------------------------
# 4.4 Behavior/Margin proxy: Discount used on repeat (repeaters only)
# -------------------------
ctrl_disc = ctrl_rep["discount_used_on_repeat"].sum()
trt_disc  = trt_rep["discount_used_on_repeat"].sum()

prop_test(
    success_a=trt_disc, n_a=trt_rep.shape[0],
    success_b=ctrl_disc, n_b=ctrl_rep.shape[0],
    label="4.4 Discount used on repeat (Treatment vs Control) — repeaters only"
)



4.1 Guardrail — Return within 14d (Treatment vs Control)
A rate: 0.1440  | B rate: 0.1416
Diff (A-B) pp: 0.24
95% CI (pp): (-1.01, 1.49)
z=0.376, p=0.706661

4.2 Time to next purchase (days) — repeaters only
Control repeaters n: 538 Treatment repeaters n: 885
       group    n       mean  median  p25   p75
0    control  538  12.120818    10.0  5.0  17.0
1  treatment  885  12.275706    11.0  6.0  17.0
Mann–Whitney U p-value: 0.4555825933125911

4.3 Repeat order value — repeaters only
       group    n       mean  median      p25     p75
0    control  538  64.043234   54.45  37.2375  80.225
1  treatment  885  65.235559   56.06  37.2800  83.640
Mann–Whitney U p-value: 0.5092555891381908

4.4 Discount used on repeat (Treatment vs Control) — repeaters only
A rate: 0.3910  | B rate: 0.4981
Diff (A-B) pp: -10.72
95% CI (pp): (-16.03, -5.41)
z=-3.958, p=0.000075


In [4]:
# Phase 5.1 — Openers / Non-openers analysis

paid = df[df.acquisition_channel == "paid"].copy()

paid_ctrl = paid[paid.variant == "control"].copy()
paid_trt  = paid[paid.variant == "treatment"].copy()

trt_open   = paid_trt[paid_trt.ugc_email_opened == 1].copy()
trt_noopen = paid_trt[paid_trt.ugc_email_opened == 0].copy()

def rate(df, col):
    return df[col].mean()

summary = pd.DataFrame({
    "group": ["control", "treatment_opened", "treatment_not_opened"],
    "n_users": [len(paid_ctrl), len(trt_open), len(trt_noopen)],
    "repeat_rate": [
        rate(paid_ctrl, "repeat_purchase_within_30d"),
        rate(trt_open, "repeat_purchase_within_30d"),
        rate(trt_noopen, "repeat_purchase_within_30d")
    ]
})

summary["repeat_rate_pp"] = summary["repeat_rate"] * 100
print(summary)


                  group  n_users  repeat_rate  repeat_rate_pp
0               control     6033     0.089176        8.917620
1      treatment_opened     2120     0.181132       18.113208
2  treatment_not_opened     3847     0.130231       13.023135


In [5]:
# Phase 5.2 — Category-level lift (Paid users only)

paid = df[df.acquisition_channel == "paid"].copy()

cat_summary = (
    paid
    .groupby(["category", "variant"])
    .agg(
        users=("user_id", "count"),
        repeat_rate=("repeat_purchase_within_30d", "mean")
    )
    .reset_index()
)

# Pivot for easier comparison
cat_pivot = (
    cat_summary
    .pivot(index="category", columns="variant", values="repeat_rate")
    .reset_index()
)

cat_pivot["lift_pp"] = (cat_pivot["treatment"] - cat_pivot["control"]) * 100
cat_pivot["control_pp"] = cat_pivot["control"] * 100
cat_pivot["treatment_pp"] = cat_pivot["treatment"] * 100

cat_pivot = cat_pivot.sort_values("lift_pp", ascending=False)

print(cat_pivot[[
    "category", "control_pp", "treatment_pp", "lift_pp"
]])


variant     category  control_pp  treatment_pp   lift_pp
4               home    8.217913     15.506958  7.289045
0            apparel    9.544707     16.500000  6.955293
3           footwear    8.586526     14.416776  5.830250
2        electronics    6.130268     10.882801  4.752532
1             beauty   12.206573     16.512915  4.306342


In [6]:
# Phase 5.3 — Region-level lift (Paid users only)

paid = df[df.acquisition_channel == "paid"].copy()

reg_summary = (
    paid
    .groupby(["region", "variant"])
    .agg(
        users=("user_id", "count"),
        repeat_rate=("repeat_purchase_within_30d", "mean")
    )
    .reset_index()
)

reg_pivot = (
    reg_summary
    .pivot(index="region", columns="variant", values="repeat_rate")
    .reset_index()
)

reg_pivot["lift_pp"] = (reg_pivot["treatment"] - reg_pivot["control"]) * 100
reg_pivot["control_pp"] = reg_pivot["control"] * 100
reg_pivot["treatment_pp"] = reg_pivot["treatment"] * 100

reg_pivot = reg_pivot.sort_values("lift_pp", ascending=False)

print(reg_pivot[[
    "region", "control_pp", "treatment_pp", "lift_pp"
]])


variant   region  control_pp  treatment_pp   lift_pp
1             NE    8.789886     16.036036  7.246150
3           West    8.268934     14.406130  6.137197
2          South    8.989424     14.186047  5.196622
0        Midwest    9.593023     14.565388  4.972364
